In [1]:
import numpy as np
import scipy
from scipy.io import loadmat
import mne, glob
from mne_features.feature_extraction import extract_features
import pandas as pd
from ipynb.fs.full.fullDataExtraction import getRawArrayData
from ipynb.fs.full.fullDataExtraction import extarctChannelNames
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
import plotly
import math
from tensorflow import keras
import dash_bootstrap_components as dbc
from ipynb.fs.full.feature_selection import compute_diffEnt
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
from dash.exceptions import PreventUpdate
import dash
import dash_mantine_components as dmc
from dash_iconify import DashIconify
import unittest
import gdown
import os

Imported 0.2.03 version. Select nrows to a small number when running on huge datasets.
output = featurewiz(dataname, target, corr_limit=0.90, verbose=2, sep=',', 
		header=0, test_data='',feature_engg='', category_encoders='',
		dask_xgboost_flag=False, nrows=None, skip_sulov=False)
Create new features via 'feature_engg' flag : ['interactions','groupby','target']



In [2]:
# make epochs of 1 second, preload loads data into memory, verbose=0 is training progress
def makeEpochs(rawArray):
    return mne.make_fixed_length_epochs(rawArray, duration=1, preload=True,verbose=0)




# extract raw EEG data from .mat file and convert to a dictionary of 
# 'mne.io.RawArray' obejcts to be analysed 
# returns dictionary - 15 mne.io.RawArray objects
# one for each key in the matfile dictionary
def getRawArrays(matfile):

    # delete keys that are not necessary for analysis
    del matfile["__header__"]
    del matfile["__version__"]
    del matfile["__globals__"]

    # empty dictionary with list of int 'indexes'
    clip_info={}
    indexs=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

    # extract first key from dicitonary 'matfile'
    # get string value of key 
    # remove last character (label)
    keyName=list(matfile.keys())[0][:-1]

    # read Excel file to get channel names for the EEG data
    # result = list of channel names
    channelNamesExtraction=extarctChannelNames(pd.read_excel("Preprocessed_EEG/channel-order.xlsx"))
    
    #creates object to store info about data
    info=mne.create_info(channelNamesExtraction,200,'eeg')

    # mne.io.RawArray object is added to the clip_info dictionary
    # integer index as the key
    for ind,i in enumerate(indexs):

        # construct key
        # raw data (string)= data (int) + index(int)
        rawData=matfile[keyName + str(i)]

        # dictionary = identifier + data 
        clip_info[ind] = mne.io.RawArray(rawData,info,verbose=0)

    return clip_info




# extract features from sliced epochs of EEG data
def ExtractFeatures(sliced_epochs,features):

    # create array to store extracted features
    combined = np.zeros((1,len(features)*62))

    # get raw data from each epoch
    # (n_epochs, n_channels, n_samples) 
    epoch_array=mne.Epochs.get_data(sliced_epochs)

    # row = single epoch
    # column = specific feature and channel
    # (n_epochs, n_features*n_channels)
    extracted_data=extract_features(epoch_array,200,features)

    # append both arrays
    combined = np.vstack((combined,extracted_data))

    # delete coz not part of the actual feature data
    combined = np.delete(combined, 0, axis=0)

    return combined

#test function
class TestExtractFeatures(unittest.TestCase):
    
    def setUp(self):
        self.features = ['mean', 'std']  # list of features
        self.epoch_array = np.random.rand(10, 62, 200)  # random epoch array for testing
        self.sliced_epochs = mne.EpochsArray(self.epoch_array, mne.create_info(62, 200, ch_types='eeg'))
    
    def test_extract_features(self):
        extracted_data = ExtractFeatures(self.sliced_epochs, self.features)
        self.assertIsInstance(extracted_data, np.ndarray)
        self.assertEqual(extracted_data.shape[0], self.epoch_array.shape[0])
        self.assertEqual(extracted_data.shape[1], len(self.features) * self.epoch_array.shape[1])

        
# creating a longer version of arr
# match the length of a longer epoch for comparison 
def extend200(arr):
    temp=[]
    for i in arr:
        for j in range(0,200):
            temp.append(i)
    return temp




#test function
class TestExtend200(unittest.TestCase):
    
    def test_extend200(self):
        arr = [1, 2, 3]
        extended_arr = extend200(arr)
        self.assertIsInstance(extended_arr, list)
        self.assertEqual(len(extended_arr), len(arr) * 200)
        self.assertEqual(extended_arr[:200], [1] * 200)
        self.assertEqual(extended_arr[200:400], [2] * 200)
        self.assertEqual(extended_arr[400:], [3] * 200)

In [3]:
# read Excel file 
channels=pd.read_excel("Preprocessed_EEG/channel-order(viz).xlsx")

# extracts the first column
channelNames=channels.iloc[:,0]

# convert object to numpy array, then to list 
channelNames=np.ndarray.tolist(pd.Series.to_numpy(channelNames))

# insert channel Fp1 to the list 
channelNames.insert(0,"Fp1")

# specifies that each channel is an EEG channel
ch_types = ['eeg'] * len(channelNames)

# create object  
info = mne.create_info(channelNames, ch_types=ch_types, sfreq=200)

# spatial arrangement of electrodes is standard 10-20 system
info.set_montage('standard_1020')

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, AF3, AF4, F7, F5, F3, F1, Fz, F2, F4, F6, F8, ...
 chs: 60 EEG
 custom_ref_applied: False
 dig: 63 items (3 Cardinal, 60 EEG)
 highpass: 0.0 Hz
 lowpass: 100.0 Hz
 meas_date: unspecified
 nchan: 60
 projs: []
 sfreq: 200.0 Hz
>

In [4]:
# 62 channels -> 60 channels 
def deleteUselessNodes(data):
    data=np.delete(data, 61, 0)
    data=np.delete(data, 57, 0)
    return data



# 3D to 2D array
def deleteUselessNodesForTest(data):
    data=np.delete(data, 0, 2)
    return data


# test case
def test_deleteUselessNodesForTest():
    # Test case 1: Ensure that function correctly removes first column of data
    input_data = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9]], [[10, 11, 12], [13, 14, 15], [16, 17, 18]]])
    expected_output = np.array([[[2, 3], [5, 6], [8, 9]], [[11, 12], [14, 15], [17, 18]]])
    assert np.array_equal(deleteUselessNodesForTest(input_data), expected_output)

    # Test case 2: Ensure that function returns empty array when input is empty
    input_data = np.array([])
    expected_output = np.array([])
    assert np.array_equal(deleteUselessNodesForTest(input_data), expected_output)

    # Test case 3: Ensure that function raises an exception when input is not a 3D array
    input_data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    try:
        deleteUselessNodesForTest(input_data)
        assert False, "Exception not raised"
    except ValueError:
        assert True

In [5]:
url = 'https://drive.google.com/uc?id=1rJn1ptWWhDagI7Au4HRZAB9ZmKqKXie6'
output_path = os.path.join(os.path.expanduser('~'), 'Downloads', 'subject.mat')

In [159]:
gdown.download(url, output_path, quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1rJn1ptWWhDagI7Au4HRZAB9ZmKqKXie6
From (redirected): https://drive.google.com/uc?id=1rJn1ptWWhDagI7Au4HRZAB9ZmKqKXie6&confirm=t&uuid=a6fc217c-5b8b-46e8-927b-774f78bec81f
To: C:\Users\Gwen\Downloads\subject.mat
100%|███████████████████████████████████████████████████████████████████████████████| 167M/167M [01:04<00:00, 2.57MB/s]


'C:\\Users\\Gwen\\Downloads\\subject.mat'

In [25]:
# load model 
model = keras.models.load_model("emotionPredictionModel")

# model file path
#downloads_path = os.path.join(os.environ['USERPROFILE'], 'Downloads')
#file_path = os.path.join(downloads_path, 'subject.mat')
subject=loadmat("data\subject.mat")

# extract raw EEG data
rawArrays=getRawArrays(subject)

# assign key name
keyName=list(subject.keys())[3][:-1]

# initialize empty lists
data=[]
evoked=[]
df=[]
y_pred=[]
extractedData=[]
transData=[]


# processes time series data (epochs),extracting features,making predictions
for i in range(1,16):

    # extract data then delete useless nodes
    data.append(deleteUselessNodes(subject[keyName+str(i)]))

    #  add transposed data to list transData
    transData.append(np.transpose(data[i-1]))

    # EvokedArray added to list evoked
    evoked.append(mne.EvokedArray(data[i-1],info))

    # convert EvokedArray to pd DataFrame
    # append DataFrame to list df
    df.append(evoked[i-1].to_data_frame())
    
    # extract fetures and append to list extractedData
    extractedData.append(ExtractFeatures(makeEpochs(rawArrays[i-1]),['hjorth_mobility', ('diffEnt',compute_diffEnt), 'rms', 'skewness']))
    
    # prediction made based on features, data, keyname 
    prediction= model.predict(extractedData[i-1])

    # convert predicted value to list of 200 elements
    y_pred.append(extend200((prediction > 0.5).astype(int).tolist()))

7/7 [==============================] - 0s 1ms/step


In [26]:
frontTup=("Frontal",['Fp1', 'Fpz', 'Fp2','AF3', 'AF4','F1', 'Fz', 'F2'])
centralTup=("Central",['FC1', 'FCz', 'FC2','C3', 'C1', 'Cz', 'C2', 'C4'])
parietalTup=("Parietal",['CP3', 'CP1', 'CPz', 'CP2', 'CP4','P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6'])
occipitalTup=("Occipital",['PO7', 'PO5', 'PO3','POz', 'PO4', 'PO6', 'PO8','O1', 'Oz', 'O2'])
leftTup=("LeftTemporal",['F4', 'F6', 'F8','FC4','FC6', 'FT8','C6','CP6', 'TP8','P8'])
rightTup=("RightTemporal",['F7', 'F5', 'F3','FT7', 'FC5', 'FC3','C5','TP7', 'CP5','P7'])
allSecList=[frontTup,centralTup,parietalTup,occipitalTup,leftTup,rightTup]
OptimalSeven=['F7','F3','Fp1','F8','PO3','P3','O2']

### Helper functions

In [27]:
# dataframe of menan value
def makeAvgframe(ser):

    # create column 'mean'
    df = pd.DataFrame(columns=['mean'])

    # calculates the mean value of each row in the input
    df['mean'] = ser.mean(axis=1)

    return df




#Test Case
class TestMakeAvgframe(unittest.TestCase):
    
    def test_make_avgframe(self):
        data = {'A': [1, 2, 3, 4], 'B': [5, 6, 7, 8], 'C': [9, 10, 11, 12]}
        ser = pd.DataFrame(data)
        avg_frame = makeAvgframe(ser)
        self.assertIsInstance(avg_frame, pd.DataFrame)
        self.assertEqual(avg_frame.shape[0], ser.shape[0])
        self.assertEqual(avg_frame.shape[1], 1)
        self.assertTrue((avg_frame['mean'] == ser.mean(axis=1)).all())
        
        

        
# build row with 3 columns for advance option dropdown 
def buildOneDdRow(options1,options2,options3):

    return dbc.Row(

        # Column 1
        [dbc.Col(
            
            # display section name
            [html.B(html.P(options1[0])),
            # display dropdown with options1
            dcc.Dropdown(options1[1],multi=True,id=options1[0])],
            width=4),

        # Column 2
        dbc.Col(
            
            # display section name
            [html.B(html.P(options2[0])),
            # display dropdown with options2
            dcc.Dropdown(options2[1],multi=True,id=options2[0])],
            width=4),

        # Column 3
        dbc.Col(

            # display section name
            [html.B(html.P(options3[0])),
            # display dropdown with options3
            dcc.Dropdown(options3[1],multi=True,id=options3[0])],    
            width=4)
        ],
         
        id = "nodeDrop"
    )


# Chart Title
def buildChartTitle(cType):

    return html.Div(

        # title for type of chart to be displayed
        cType + " of Current Node Readings",
        className = "title"
    )




# Display emotion and activities
def decipherEmotion(time_instance,ddChoice):
    
    # don't display if time not chosen
    if time_instance != None:
        
        # Positive
        if (y_pred[ddChoice][time_instance] == [1,0,0]):

            emotion = "positive",
            row = html.Div([
                
                # GIF
                html.Img(src="assets/positive.gif"),

                # suggested activities
                html.H5(["Here are some activities to enhance your mood:"]),
                html.P([html.Ul(),"Engage in physical activity: going for a run or a hike.",html.Br(),
                        html.Ul(),"Do something creative: painting, drawing, or playing an instrument."])
                
            ])      
             

        # Neutral    
        elif (y_pred[ddChoice][time_instance] == [0,1,0]):

            emotion = "neutral"
            row = html.Div([
                    
                # GIF
                html.Img(src="assets/neutral.gif"),

                # suggested activities
                html.H5(["Here are some activities to lift your mood:"]),
                html.P([html.Ul(),"Learn something new: it is a great way to challenge yourself and feel more positive!",html.Br(),
                        html.Ul(),"Engage in a hobby: reading, gardening, or cooking can be a great way to relax and enjoy yourself."])
            ])  
            
        
        # Negative 
        else:
            emotion="negative" 
            row = html.Div([
                
                # GIF 
                html.Img(src="assets/negative.gif"),

                # suggested activities    
                html.H5(["Here are some activities to boost your mood:"]),
                html.P([html.Ul(),"Connect with others: Reach out to a friend or family member!",html.Br(),
                        html.Ul(),"Take a break: help us to recharge from burntout and feel more cheerful!"])
            ])  
            
           
        emotionDiv = html.H2(emotion)
        
        
        return emotionDiv, row 
    
    else:
        return html.H4("No time frame has been selected yet")
    

    
    
def makeValueMatrix(currentAnalysed):

    # return 2D numpy array 'result'
    result=pd.read_excel("Preprocessed_EEG/channel-order(topo).xlsx").to_numpy().astype(np.float32)
    
    #tracker value
    k = 0

    # check if its a number
    for i in range(0,result.shape[0]):

        for j in range (0,result.shape[1]):
            
            # if its a number then replace with currentAnalysed
            if not(math.isnan(result[i][j])):

                result[i][j]=currentAnalysed[k]
                k+=1
    
    return result




# create empty plotly graph 
def returnEmptyGraph():

    # create empty scatter plot 
    fig = go.Figure(go.Scatter(x=[], y = []))

    # make graph background same as dashboard background
    fig.update_layout(plot_bgcolor='#18242B',paper_bgcolor='#18242B',)

    # hide axes, lines and mode bar
    fig.update_xaxes(showgrid = False, showticklabels = False, zeroline=False)
    fig.update_yaxes(showgrid = False, showticklabels = False, zeroline=False)
    empty=dcc.Graph(figure=fig,config = {'displayModeBar': False})

    return empty




#test case
class TestReturnEmptyGraph(unittest.TestCase):
    
    def test_return_empty_graph(self):
        empty = returnEmptyGraph()
        self.assertIsInstance(empty, dcc.Graph)
        self.assertIsInstance(empty.figure, go.Figure)
        self.assertEqual(empty.figure['data'][0]['x'], ())
        self.assertEqual(empty.figure['data'][0]['y'], ())
        self.assertEqual(empty.figure['layout']['plot_bgcolor'], '#18242B')
        self.assertEqual(empty.figure['layout']['paper_bgcolor'], '#18242B')
        self.assertFalse(empty.figure['layout']['xaxis']['showgrid'])
        self.assertFalse(empty.figure['layout']['xaxis']['showticklabels'])
        self.assertFalse(empty.figure['layout']['xaxis']['zeroline'])
        self.assertFalse(empty.figure['layout']['yaxis']['showgrid'])
        self.assertFalse(empty.figure['layout']['yaxis']['showticklabels'])
        self.assertFalse(empty.figure['layout']['yaxis']['zeroline'])
        self.assertFalse(empty.config['displayModeBar'])

        
# axis labels and colours        
def getAxisNames():

    return dict(x="Channel Names", y="Voltage", color="Channel Names")




#test case
class TestGetAxisNames(unittest.TestCase):
    
    def test_get_axis_names(self):
        expected_output = {"x": "Channel Names", "y": "Voltage", "color": "Channel Names"}
        output = getAxisNames()
        self.assertEqual(output, expected_output)


# takes in the type of chart
# takes in list of 2-tuples - section name, list of channel names
def buildCharts(type,list):
    
    # create an empty list to store the results
    res = []
    
    # iterate over each element in the input list
    for i in list:
        
        # append a new row to the results list, with a single column
        # the id of the column is created by concatenating the type of chart and section name
        res.append(dbc.Row(dbc.Col(id=(type+i[0]))))
        
    return res




# test case
def test_build_charts():
    # Test case 1: Ensure that function returns correct output for non-empty input
    input_list = [("a", 1), ("b", 2), ("c", 3)]
    expected_output = [
        dbc.Row(dbc.Col(id="a1")),
        dbc.Row(dbc.Col(id="b2")),
        dbc.Row(dbc.Col(id="c3"))
    ]
    assert buildCharts("", input_list) == expected_output

    # Test case 2: Ensure that function returns empty list for empty input
    input_list = []
    expected_output = []
    assert buildCharts("", input_list) == expected_output

    # Test case 3: Ensure that function raises an exception when input is not a list
    input_list = "not a list"
    with pytest.raises(TypeError):
        buildCharts("", input_list)




## Main component definition

### slider/input

In [28]:
# ==================== Slider ====================

# Initialize the variable choice with 0
choice = 0

slider = html.Div(
        
        [dcc.Slider(
    
            # Set the minimum value of the slider to 0
            min=0,
            
            # Set the maximum value of the slider to the number of columns in the dataframe
            # max vlue based on different clips (index [choice]) 
            max=data[choice].shape[1]-1,

            # Set the step value of the slider to 1
            step=1,

            # no marking on slider
            marks=None,
            
            # Set the tooltip to not always be visible
            tooltip={"always_visible": False},
            
            id = "time"
        )]
)




# ==================== Time Label and Input ====================

# Column for Time Label
labelCol = dbc.Col(
    
            html.Div([html.H3("Select time frame")])
)




# Column for Input Field
inputCol = dbc.Col(
    
            dcc.Input(
    
                # accept numerical values 
                type="number",
        
                # Set the minimum value of the slider to 0
                min=0,
            
                # Set the maximum value of the slider to the number of columns in the dataframe
                # max vlue based on different clips (index [choice]) 
                max=data[choice].shape[1]-1,

                placeholder="Input time frame",
       
                id="inputTime",
            ),
)




# ==================== Time Field ====================

timeField = html.Div(

            # 1st row: label and input field 
            [dbc.Row([labelCol, inputCol,]),
    
            # 2nd row: slider
            dbc.Row([slider])
            ], 

            #optional========================
            id ="timeField"
            
)

### Advanced option selection

In [29]:
# ==================== Switch ====================

# row for switch for advance option
advancedOptionRow=dbc.Row(
    
            dmc.Switch(
                
                id="moreOptionsChecklist",

                label=html.Span("More Options", style={'color': '#95969A','font-weight':'bold'}),

                # make the switch persist its state when the page is refreshed
                persistence_type='local',

                # switch will start in "off" position
                checked=False,
                
                color="teal"),
                    
            id='moreOptionsSel'
)



# ==================== Node Selection ====================

# labels and rows for node selection
dropDownLabel = dbc.Row(
    
            [html.H3("Node selection")],

            id = "nodeLabel"
)




dropDownRow = dbc.Row(

            # 1st row: label  
            [dbc.Row([dropDownLabel]),
    
            # 2nd and 3rd row: section label and dropdown
            dbc.Row([buildOneDdRow(frontTup,centralTup,parietalTup,),
                     buildOneDdRow(occipitalTup,leftTup,rightTup)])
            ],
            
             id = "nodeSectionLabel"
)


   

### clip select

In [30]:
# ==================== Clip Label ====================

clipSelectLabel = html.Div(
    
            [html.H3(["select clip"])]
)




# ==================== Clip Dropdown ====================

# create 15 labels for the clips
clipLabels = []

for i in range(1,16):
    clipLabels.append(str(i))




# dropdown
clipSelectDD = dbc.Row([
    
            # 1st column: label  
            dbc.Col(clipSelectLabel),

            # 2nd column: dropdown
            dbc.Col(dcc.Dropdown(
                clipLabels,
                placeholder="1",
                id="clipDrop"
            ),
        
    )],
    
)


# ==================== Clip Field ====================
clipField = html.Div(
    
            # 1st row: clip label and dropdown
            [clipSelectDD,

            #2nd row: switch
            advancedOptionRow,
            dbc.Row(id="moreOptions")],
    
            id = "clipField",
            
)

## value pick tab

In [31]:
# ==================== User Input and Options ====================

optionStack = dbc.Col(
    
            [dbc.Stack([timeField, clipField])],
   
            id='optionPick'
)




# ==================== Emotion Display ====================

emotionCol = dbc.Col(
    
            id="emotion"   
)




# ==================== Value Tab ====================

valSec=dbc.Row([
    
        optionStack,
        emotionCol],
    
        style={'margin-left':'1px'}
)

## charts

In [32]:
# ==================== Heat Map ====================

heatMap = dbc.Col([
    
    # Title of chart
    buildChartTitle("Heat Map"),
    
    # column for heatmap
    dbc.Col(id='heatmap'
           )],
    
    
)




# ==================== Bar Chart ====================

advancedBarCharts = dbc.Col([
    
    # Title of chart
    buildChartTitle("Bar Charts"),
    
    # column for bar chart
    dbc.Col(buildCharts('barChart',allSecList),
        
            id="barChartContainer",
        
    )
],
)




# ==================== GFP Analysis ====================

advancedLineCharts=dbc.Row([
    
    # Title of chart
    buildChartTitle("Global field power (GFP)"),
    
    # column for line chart
    dbc.Col(
        buildCharts('lineChart',allSecList),
        id="lineChartContainer",
        #width=6
    ),
    
    # column for GFP Analysis
    dbc.Col(
        id="GFPContainer",
        #width=6,
    )
])

## charts section

### Heatmap section

In [33]:
# ==================== Heat Map Tab ====================

heatMapSec=dbc.Row([
    
    # 1st column: heatmap
    dbc.Col(
        [heatMap],
        id='heatMapDisplay',
        width=8,
    ),
    
    # 2nd column: description
    dbc.Col(
        [
            html.H4(["Description:"]),
            html.P(["A heatmap is a tidy and understandable graphical representation of data that uses color-coding to represent different values. ",
                    "In the context of EEG signals, a heatmap can be used to visualize the activity of different regions of the brain over time. ",
                    html.Br(), html.Br(),
                    "The heatmap is typically generated by dividing the EEG signal into a grid of small segments and then calculating the average amplitude or power of each segment. ",
                    "The resulting values are then mapped to a color scale, with warmer colors (e.g. red, orange, yellow) representing higher values and cooler colors (e.g. purple) representing lower values. ",
                    "By examining the heatmap, we can gain insights into the spatial and temporal patterns of brain activity."
             ])
        ],
        id='heatMapDes',
        width=4
    )
],
    className = "section"
)

### Bar Chart Section

In [34]:
# ==================== Bar Chart Tab ====================

barChartSec=dbc.Row([
    
    # 1st column: bar charts
    dbc.Col(
        [
            advancedBarCharts,
        ],
        id='barChartDisplay',
        width=8
    ),
    
    # 2nd column: description
    dbc.Col(
        [
            html.H4(["Description:"]),
            html.P(["In a bar chart, each bar represents the strength of a particular channel in the EEG signal. ",
                    "The length of each bar reflects the value of the data, with longer bars indicating higher values and shorter bars indicating lower values. ",
                    " It is particularly useful for identifying channels with high or low activity, as well as for comparing the activity levels of different channels. ",
                    html.Br(), html.Br(),
                    "Emotions are known to be associated with specific patterns of brain activity, and researchers have identified specific channels in the EEG signal that are particularly relevant for emotion prediction. ",
                    html.Br(), html.Br(),
                    "By analyzing EEG data using bar charts, researchers can identify patterns of activity in these key channels that are associated with different emotions. ",
                    "For example, increased activity in the left frontal region of the brain is associated with positive emotions such as happiness and excitement, while increased activity in the right frontal region is associated with negative emotions such as fear and anxiety. "
                    
             ])
        ],
        id='barChartDes',
        width=4
    )
],
    className = "section"
)

### GFP section

In [35]:
# ==================== GFP Tab ====================

GFPSec=dbc.Row([
    dbc.Row([   
        advancedLineCharts
        ],
        id='GFPDisplay'
    ),
    dbc.Row(
        id = "GFPDes",
    )
], className = "section")

### Layout

In [36]:
external_stylesheets = ["assets/style.css", dbc.themes.LUX]
app = JupyterDash(__name__, external_stylesheets = external_stylesheets)

app.layout = dbc.Container([
    
    dcc.Store(id='timeStore'),
    dcc.Store(id='clipChoice'),
    dcc.Store(id='frontChoice'),
    dcc.Store(id='centralChoice'),
    dcc.Store(id='parietalChoice'),
    dcc.Store(id='occipitalChoice'),
    dcc.Store(id='leftChoice'),
    dcc.Store(id='rightChoice'),
    dcc.Store(id='advancedChoice'),
    
    dbc.Row([
        dbc.Row(
            html.H1('Emotion Prediction With MNE'),
            style={'padding-top':'1rem','padding-bottom':'1rem'}
        ),
        dbc.Row(
            dmc.Tabs(
                    [
                        dmc.TabsList(
                            
                            
                            [
                                
                                dmc.Tab('VALUES', icon=DashIconify(icon="material-symbols:display-settings-rounded"), value='valSec', className='tab',style={'color': 'white'}),
                                dmc.Tab('HEAT MAP', icon=DashIconify(icon="mdi:head"), value='chartSec', className='tab',style={'color': 'white'}),
                                dmc.Tab('BAR CHARTS', icon=DashIconify(icon="ion:bar-chart-sharp"),value='barSec', className='tab',style={'color': 'white'}),
                                dmc.Tab('GFP ANALYSIS', icon=DashIconify(icon="codicon:graph-line"),value='GFPSec', className='tab',style={'color': 'white'}),   
                                
                            ],
                         
                            grow = True,
                            
                            #style= {
                            #'backgroundColor': '#18242B',
                            #'borderColor': '#23262E',
                            #'display': 'flex',
                            ##'flex-direction': 'row',
                           # 'alignItems': 'center',
                            #'justifyContent': 'center'
                        #},
                            
                         
                            
                        )                         
                    ],
                    id="tabSel",
                    value='valSec',
                    color="teal"
                )
            ),
        dbc.Row(id='mainContent',children=valSec),
    ]),
])

## Callbacks

### advanced option callbacks

In [37]:
#callback to update advanced options
@app.callback(
    Output("moreOptions", "children"), 
    Input("moreOptionsChecklist", "checked"),
    Input('advancedChoice','data')
)
def showAdvancedOption(checked, savedData):
    if checked or savedData:
        return dropDownRow
    else:
        return " "
    
#callback to store the value of the advanced option switch
@app.callback(
    Output('advancedChoice','data'),
    Input("moreOptionsChecklist", "checked"),
)
def storeAdvancedChoice(checked):
    if checked == None or False:
        raise PreventUpdate
    else:
        return checked

#callback to dynamically update the value section scroll bar
@app.callback(
    Output('optionPick','style'),
    Input("moreOptionsChecklist", "checked")
)
def updateScroll(checked):
    if (checked):
        return {
                'display':'flex', 
                'justifyContent':'center',
                'padding-top':'2rem',
                'height':'75vh',
                'overflow-y':'scroll',
                'overflow-x':'hidden'
                }
    else:
        return {
                'display':'flex', 
                'justifyContent':'center',
                'padding-top':'2rem',
                'height':'75vh',
                'overflow-y':'hidden',
                'overflow-x':'hidden'
                }
#testsuite
class TestUpdateScroll(unittest.TestCase):

    def test_scroll_enabled(self):
        result = updateScroll(True)
        self.assertEqual(result['overflow-y'], 'scroll')
        self.assertEqual(result['overflow-x'], 'hidden')

    def test_scroll_disabled(self):
        result = updateScroll(False)
        self.assertEqual(result['overflow-y'], 'hidden')
        self.assertEqual(result['overflow-x'], 'hidden')

### dropdown callbacks

In [38]:
#callback to store the clip choice
@app.callback(
    Output('clipChoice','data'),
    Input('clipDrop','value')
)
def filterClip(ddChoice):
    if not ddChoice:
        return 0
    return int(ddChoice)-1

class TestFilterClip(unittest.TestCase):

    def test_no_dd_choice(self):
        result = filterClip(None)
        self.assertEqual(result, 0)

    def test_dd_choice(self):
        result = filterClip("2")
        self.assertEqual(result, 1)

#Clip choice
@app.callback(
    Output('clipDrop','value'),
    Input('clipDrop','value'),
    Input('clipChoice','data')
)
def accessStoredClip(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default
    
class TestAccessStoredClip(unittest.TestCase):
    def test_default_none_and_saved_data_none(self):
        self.assertIsNone(accessStoredClip(None, None))
    
    def test_default_none_and_saved_data_not_none(self):
        saved_data = "Hello, world!"
        self.assertEqual(accessStoredClip(None, saved_data), saved_data)
    
    def test_default_not_none_and_saved_data_none(self):
        default = 42
        self.assertEqual(accessStoredClip(default, None), default)

#callback to store frontal node choices
@app.callback(
    Output('frontChoice','data'),
    Input(frontTup[0],'value')
)
def filterFrontal(choices):
    if not choices:
        return frontTup[1]
    return choices

class TestFilterFrontal(unittest.TestCase):

    def test_empty_choices(self):
        self.assertEqual(filterFrontal([]), frontTup[1])

    def test_nonempty_choices(self):
        choices = [1, 2, 3]
        self.assertEqual(filterFrontal(choices), choices)
        
    def test_empty_choices(self):
        self.assertEqual(filterFrontal(None), frontTup[1])

#callback to store central node choices
@app.callback(
    Output('centralChoice','data'),
    Input(centralTup[0],'value')
)
def filterCentral(choices):
    if not choices:
        return centralTup[1]
    return choices

class TestFilterCentral(unittest.TestCase):
    
    def test_empty_choices(self):
        self.assertEqual(filterCentral([]), centralTup[1])
                         
    def test_nonempty_choices(self):
        choices = [1, 2, 3]
        self.assertEqual(filterCentral(choices), choices)
    
    def test_empty_choices(self):
        self.assertEqual(filterParietal(None), parietalTup[1])

#callback to store parietal node choices
@app.callback(
    Output('parietalChoice','data'),
    Input(parietalTup[0],'value')
)
def filterParietal(choices):
    if not choices:
        return parietalTup[1]
    return choices

class TestFilterParietal(unittest.TestCase):
    
    def test_empty_choices(self):
        self.assertEqual(filterParietal([]), parietalTup[1])
        
    def test_nonempty_choices(self):
        choices = [1, 2, 3]
        self.assertEqual(filterParietal(choices), choices)
        
    def test_empty_choices(self):
        self.assertEqual(filterParietal(None), parietalTup[1])

#callback to store occipital node choices
@app.callback(
    Output('occipitalChoice','data'),
    Input(occipitalTup[0],'value')
)
def filterOccipital(choices):
    if not choices:
        return occipitalTup[1]
    return choices

class TestFilterOccipital(unittest.TestCase):
    
    def test_empty_choices(self):
        self.assertEqual(filterOccipital([]), occipitalTup[1])
        
    def test_nonempty_choices(self):
        choices = [1, 2, 3]
        self.assertEqual(filterOccipital(choices), choices)
    
    def test_empty_choices(self):
        self.assertEqual(filterOccipital(None), occipitalTup[1])

#callback to store left node choices
@app.callback(
    Output('leftChoice','data'),
    Input(leftTup[0],'value')
)
def filterLeft(choices):
    if not choices:
        return leftTup[1]
    return choices

class TestFilterLeft(unittest.TestCase):
    
    def test_empty_choices(self):
        self.assertEqual(filterLeft([]), leftTup[1])
        
    def test_nonempty_choices(self):
        choices = [1, 2, 3]
        self.assertEqual(filterLeft(choices), choices)
        
    def test_empty_choices(self):
        self.assertEqual(filterLeft(None), leftTup[1])


#callback to store right node choices
@app.callback(
    Output('rightChoice','data'),
    Input(rightTup[0],'value')
)
def filterRight(choices):
    if not choices:
        return rightTup[1]
    return choices

class TestFilterRight(unittest.TestCase):
    def test_empty_input_returns_righttup(self):
        self.assertEqual(filterRight([]), rightTup[1])

    def test_nonempty_input_returns_choices(self):
        choices = [1, 2, 3]
        self.assertEqual(filterRight(choices), choices)

    def test_input_is_none_returns_righttup(self):
        self.assertEqual(filterRight(None), rightTup[1])


#Frontal channel choices
@app.callback(
    Output(frontTup[0],'value'),
    Input(frontTup[0],'value'),
    Input('frontChoice','data')
)
def accessStoredFrontal(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default
    
class TestAccessStoredFrontal(unittest.TestCase):
    def test_default_is_none_and_savedData_is_not_none(self):
        default = None
        savedData = [1, 2, 3]
        result = accessStoredFrontal(default, savedData)
        self.assertEqual(result, savedData)

    def test_default_is_not_none_and_savedData_is_not_none(self):
        default = "hello"
        savedData = [1, 2, 3]
        result = accessStoredFrontal(default, savedData)
        self.assertEqual(result, default)

    def test_default_is_none_and_savedData_is_none(self):
        default = None
        savedData = None
        result = accessStoredFrontal(default, savedData)
        self.assertEqual(result, None)

    def test_default_is_not_none_and_savedData_is_none(self):
        default = "hello"
        savedData = None
        result = accessStoredFrontal(default, savedData)
        self.assertEqual(result, default)

#Central channel choices
@app.callback(
    Output(centralTup[0],'value'),
    Input(centralTup[0],'value'),
    Input('centralChoice','data')
)
def accessStoredCentral(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default
    
class TestAccessStoredCentral(unittest.TestCase):
    def test_default_is_none_and_savedData_is_not_none(self):
        default = None
        savedData = [1, 2, 3]
        result = accessStoredCentral(default, savedData)
        self.assertEqual(result, savedData)

    def test_default_is_not_none_and_savedData_is_not_none(self):
        default = "hello"
        savedData = [1, 2, 3]
        result = accessStoredCentral(default, savedData)
        self.assertEqual(result, default)

    def test_default_is_none_and_savedData_is_none(self):
        default = None
        savedData = None
        result = accessStoredCentral(default, savedData)
        self.assertEqual(result, None)

    def test_default_is_not_none_and_savedData_is_none(self):
        default = "hello"
        savedData = None
        result = accessStoredCentral(default, savedData)
        self.assertEqual(result, default)
    
#Parietal channel choice
@app.callback(
    Output(parietalTup[0],'value'),
    Input(parietalTup[0],'value'),
    Input('parietalChoice','data')
)
def accessStoredParietal(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default
    
class TestAccessStoredParietal(unittest.TestCase):
    def test_default_is_none_and_savedData_is_not_none(self):
        default = None
        savedData = [1, 2, 3]
        result = accessStoredParietal(default, savedData)
        self.assertEqual(result, savedData)

    def test_default_is_not_none_and_savedData_is_not_none(self):
        default = "hello"
        savedData = [1, 2, 3]
        result = accessStoredParietal(default, savedData)
        self.assertEqual(result, default)

    def test_default_is_none_and_savedData_is_none(self):
        default = None
        savedData = None
        result = accessStoredParietal(default, savedData)
        self.assertEqual(result, None)

    def test_default_is_not_none_and_savedData_is_none(self):
        default = "hello"
        savedData = None
        result = accessStoredParietal(default, savedData)
        self.assertEqual(result, default)
        
    
#Occipital channel choice
@app.callback(
    Output(occipitalTup[0],'value'),
    Input(occipitalTup[0],'value'),
    Input('occipitalChoice','data')
)
def accessStoredOccipital(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default

class TestAccessStoredOccipital(unittest.TestCase):
    def test_default_is_none_and_savedData_is_not_none(self):
        default = None
        savedData = [1, 2, 3]
        result = accessStoredOccipital(default, savedData)
        self.assertEqual(result, savedData)

    def test_default_is_not_none_and_savedData_is_not_none(self):
        default = "hello"
        savedData = [1, 2, 3]
        result = accessStoredOccipital(default, savedData)
        self.assertEqual(result, default)

    def test_default_is_none_and_savedData_is_none(self):
        default = None
        savedData = None
        result = accessStoredOccipital(default, savedData)
        self.assertEqual(result, None)

    def test_default_is_not_none_and_savedData_is_none(self):
        default = "hello"
        savedData = None
        result = accessStoredOccipital(default, savedData)
        self.assertEqual(result, default)
    
#left channel choice
@app.callback(
    Output(leftTup[0],'value'),
    Input(leftTup[0],'value'),
    Input('leftChoice','data')
)
def accessStoredLeft(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default
    
class TestAccessStoredOccipital(unittest.TestCase):
    def test_default_is_none_and_savedData_is_not_none(self):
        default = None
        savedData = [1, 2, 3]
        result = accessStoredOccipital(default, savedData)
        self.assertEqual(result, savedData)

    def test_default_is_not_none_and_savedData_is_not_none(self):
        default = "hello"
        savedData = [1, 2, 3]
        result = accessStoredOccipital(default, savedData)
        self.assertEqual(result, default)

    def test_default_is_none_and_savedData_is_none(self):
        default = None
        savedData = None
        result = accessStoredOccipital(default, savedData)
        self.assertEqual(result, None)

    def test_default_is_not_none_and_savedData_is_none(self):
        default = "hello"
        savedData = None
        result = accessStoredOccipital(default, savedData)
        self.assertEqual(result, default)
    
#Right channel choice
@app.callback(
    Output(rightTup[0],'value'),
    Input(rightTup[0],'value'),
    Input('rightChoice','data')
)
def accessStoredRight(default, savedData):
    if default == None:
        if savedData != None:
            return savedData
    else:
        return default
    
class TestAccessStoredRight(unittest.TestCase):
    def test_default_is_none_and_savedData_is_not_none(self):
        default = None
        savedData = [1, 2, 3]
        result = accessStoredRight(default, savedData)
        self.assertEqual(result, savedData)

    def test_default_is_not_none_and_savedData_is_not_none(self):
        default = "hello"
        savedData = [1, 2, 3]
        result = accessStoredRight(default, savedData)
        self.assertEqual(result, default)

    def test_default_is_none_and_savedData_is_none(self):
        default = None
        savedData = None
        result = accessStoredRight(default, savedData)
        self.assertEqual(result, None)

    def test_default_is_not_none_and_savedData_is_none(self):
        default = "hello"
        savedData = None
        result = accessStoredRight(default, savedData)
        self.assertEqual(result, default)

### slider/input callbacks

In [39]:
#callback to update slider/input max
@app.callback(
    Output('time','max'),
    Output('inputTime','max'),
    Input('clipDrop','value')
)

def updateMax(ddChoice):
    if not ddChoice:
        return transData[0].shape[0],transData[0].shape[0]
    return transData[int(ddChoice)-1].shape[0],transData[int(ddChoice)-1].shape[0]

class TestUpdateMax(unittest.TestCase):

    def test_with_empty_ddChoice(self):
        ddChoice = ""
        expected_output = (transData[0].shape[0], transData[0].shape[0])
        self.assertEqual(updateMax(ddChoice), expected_output)

    def test_with_valid_ddChoice(self):
        ddChoice = "2"
        expected_output = (transData[1].shape[0], transData[1].shape[0])
        self.assertEqual(updateMax(ddChoice), expected_output)


#callback to store time
@app.callback(
    Output('timeStore','data'),
    Input('inputTime','value')
)
def storeTime(data):
    if data == None:
        raise PreventUpdate
    else:
        return data
#test case
class TestStoreTime(unittest.TestCase):
    
    def test_store_time_with_data(self):
        # Test case where data is not None
        data = "2022-03-06 15:30:00"
        expected_output = data
        
        output = storeTime(data)
        
        self.assertEqual(output, expected_output)
        
    def test_store_time_with_none(self):
        # Test case where data is None
        data = None
        
        with self.assertRaises(PreventUpdate):
            storeTime(data)
    
#callback to update the slider and input
@app.callback(
    Output("inputTime","value"),
    Output("time","value"),
    Input("inputTime","value"),
    Input("time","value"),
    Input("timeStore","data")
)
def updateSldierOrInput(inputVal,sliderVal,timeStore):
    ctx = dash.callback_context
    trigger_id = ctx.triggered[0]["prop_id"].split(".")[0]
    value1 = inputVal if trigger_id == "inputTime" else sliderVal
    value2 = timeStore
    if value1 == None:
        if value2 != None:
            return value2, value2
    else:
        return value1, value1


### tab callbacks

In [40]:
#callback to render out the tabs
@app.callback(
    Output('mainContent','children'),
    Input('tabSel','value')
)
def renderContent(tab):
    if tab=='valSec':
        return valSec
    elif tab == 'barSec':
        return barChartSec
    elif tab == 'GFPSec':
        return GFPSec
    else:
        return heatMapSec
#test case
class TestRenderContent(unittest.TestCase):
    
    def test_valSec(self):
        result = renderContent('valSec')
        self.assertEqual(result, valSec)
    
    def test_barSec(self):
        result = renderContent('barSec')
        self.assertEqual(result, barChartSec)
    
    def test_GFPSec(self):
        result = renderContent('GFPSec')
        self.assertEqual(result, GFPSec)
    
    def test_heatMapSec(self):
        result = renderContent('heatMapSec')
        self.assertEqual(result, heatMapSec)
    

### emotion display callback

In [41]:
#callback to update the emotions display
@app.callback(
    Output('emotion','children'),
    Input('inputTime','value'),
    Input('timeStore','data'),
    Input('clipDrop','value')
)
def updateEmotions(curTime,storedTime,ddChoice):
    if (curTime!= None) and (ddChoice != None):
        return decipherEmotion(curTime,int(ddChoice)-1)
    else:
        return decipherEmotion(storedTime,0)


### chart callbacks

In [42]:
#callback to update heatmap
@app.callback(
    Output('heatmap','children'),
    Input('timeStore','data'),
    Input('clipChoice','data'),
)
def updateHeatMap(time_instance,ddChoice):
    if (time_instance is None):
        raise PreventUpdate
    else:
        axis_names=getAxisNames()
        
        if (ddChoice is None):
            currentDf=df[0]
        else:
            currentDf=df[ddChoice]
            
        heatMap = px.imshow(makeValueMatrix(transData[ddChoice][time_instance]))
        heatMap.update_xaxes(showticklabels=False)
        heatMap.update_yaxes(showticklabels=False)
        heatMap.update_layout(xaxis=dict(scaleanchor='y', constrain='domain'),
                              coloraxis=dict(colorbar=dict(orientation='h',thickness=20,len=1.0)),
                              plot_bgcolor='#18242B',
                              paper_bgcolor='#18242B',
                              font_color='#95969A')
        heatMapFig=dcc.Graph(figure=heatMap)
        
        return heatMapFig

#callback to update bar charts
@app.callback(
    Output("barChartFrontal", "children"),
    Output("barChartCentral", "children"),
    Output("barChartParietal", "children"),
    Output("barChartOccipital", "children"),
    Output("barChartLeftTemporal", "children"),
    Output("barChartRightTemporal", "children"),
    Output("barChartContainer","style"),
    Input('timeStore','data'),
    Input('frontChoice','data'),
    Input('centralChoice','data'),
    Input('parietalChoice','data'),
    Input('occipitalChoice','data'),
    Input('leftChoice','data'),
    Input('rightChoice','data'),
    Input('clipChoice','data'),
    Input("advancedChoice", "data")
)
def updateChartSec(time_instance,front,central,parietal,occipital,left,right,ddChoice,checked):
    if (time_instance is None):
        raise PreventUpdate
    else:
        axis_names=getAxisNames()
        
        if (ddChoice is None):
            currentDf=df[0]
        else:
            currentDf=df[ddChoice]
        
        #indicates that the advanced option is not selected
        if not checked:
            ser=currentDf[OptimalSeven].iloc[time_instance]
            oneBar=px.bar(x=ser.axes[0].tolist(),y=ser.values,color=ser.axes[0].tolist())
            oneBar.update_layout(xaxis_title=axis_names['x'],
                                 yaxis_title=axis_names['y'],
                                 legend_title=axis_names['color'],
                                 plot_bgcolor='#18242B',
                                 paper_bgcolor='#18242B',
                                 font_color='#95969A'
                                )
            oneBarGraph=dcc.Graph(figure=oneBar)
            style={'height':'71.5vh','overflow-x': 'hidden','overflow-y': 'hidden'}
            return oneBarGraph,returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),style
        
        
        
        ser=currentDf[front].iloc[time_instance]
        frontalBar=px.bar(x=front,y=ser.values,color=front,title="Frontal",labels=axis_names)
        frontalBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B', font_color='#95969A')
        barChartFrontal= dcc.Graph(figure=frontalBar)
        
    
        ser=currentDf[central].iloc[time_instance]
        centralBar=px.bar(x=central,y=ser.values,color=central,title="Central",labels=axis_names)
        centralBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
        barChartCentral= dcc.Graph(figure=centralBar)
        
        
        ser=currentDf[parietal].iloc[time_instance]
        parietalBar=px.bar(x=parietal,y=ser.values,color=parietal,title="Parietal",labels=axis_names)
        parietalBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B', font_color='#95969A')
        barChartParietal= dcc.Graph(figure=parietalBar)
        
        
        ser=currentDf[occipital].iloc[time_instance]
        occipitalBar=px.bar(x=occipital,y=ser.values,color=occipital,title="Occipital",labels=axis_names)
        occipitalBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B', font_color='#95969A')
        barChartOccipital= dcc.Graph(figure=occipitalBar)
    
    
    
        ser=currentDf[left].iloc[time_instance]
        ltBar=px.bar(x=left,y=ser.values,color=left,title="Left Temporal",labels=axis_names)
        ltBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B', font_color='#95969A')
        barChartLeftTemporal= dcc.Graph(figure=ltBar)
    
    
    
        ser=currentDf[right].iloc[time_instance]
        rfBar=px.bar(x=right,y=ser.values,color=right,title="Right Temporal",labels=axis_names)
        rfBar.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
        barChartRightFrontal= dcc.Graph(figure=rfBar)
        
        style={'height':'71.5vh','overflow-x': 'hidden','overflow-y': 'scroll'}
        
        return barChartFrontal,barChartCentral,barChartParietal,barChartOccipital,barChartLeftTemporal,barChartRightFrontal,style
    
#callback to update the GFP/line charts
@app.callback(
    Output("lineChartFrontal", "children"),
    Output("lineChartCentral", "children"),
    Output("lineChartParietal", "children"),
    Output("lineChartOccipital", "children"),
    Output("lineChartLeftTemporal", "children"),
    Output("lineChartRightTemporal", "children"),
    Output("GFPContainer","children"),
    Output("lineChartContainer","style"),
    Input('timeStore','data'),
    Input('frontChoice','data'),
    Input('centralChoice','data'),
    Input('parietalChoice','data'),
    Input('occipitalChoice','data'),
    Input('leftChoice','data'),
    Input('rightChoice','data'),
    Input('clipChoice','data'),
    Input("advancedChoice", "data")
)
def outPutGFPCharts(time_instance,front,central,parietal,occipital,left,right,ddChoice,checked):
    axis_names=getAxisNames()
    if (time_instance is None):
        raise PreventUpdate
    else:
        if (ddChoice is None):
            currentDf=df[0]
        else:
            currentDf=df[ddChoice]
            
        maxTf=time_instance+100
        minTf=time_instance-100
        
        if minTf<0:
            minTf=0
        if maxTf>=data[ddChoice].shape[1]:
            maxTf=data[ddChoice].shape[1]-1
        
        GFP=evoked[ddChoice].data.std(axis=0, ddof=0)[minTf:maxTf]
        GFPfigure=px.line(GFP)
        GFPfigure.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
        GFPFig=dcc.Graph(figure=GFPfigure)
        
        if not checked:
            ser=currentDf[OptimalSeven].loc[minTf:maxTf:1]
            avgFrame=makeAvgframe(ser)
            readAndAvg=pd.concat([ser,avgFrame])
            oneLine=px.line(readAndAvg,labels=axis_names)
            oneLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            optimalChart=dcc.Graph(figure=oneLine);
            
            style={'height':'71.5vh','overflow-x': 'hidden','overflow-y': 'hidden'}
            
            return optimalChart,returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),returnEmptyGraph(),GFPFig,style
        
        else:
            frontalLine=px.line(currentDf[front].loc[minTf:maxTf:1])
            frontalLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartFrontal= dcc.Graph(figure=frontalLine)
            
            
            centralLine=px.line(currentDf[central].loc[minTf:maxTf:1])
            centralLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartCentral= dcc.Graph(figure=centralLine)
    
            parietalLine=px.line(currentDf[central].loc[minTf:maxTf:1])
            parietalLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartParietal= dcc.Graph(figure=parietalLine)
            
            
            occipitalLine=px.line(currentDf[occipital].loc[minTf:maxTf:1])
            occipitalLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartOccipital= dcc.Graph(figure=occipitalLine)
            
            
            ltLine=px.line(currentDf[left].loc[minTf:maxTf:1])
            ltLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartLeftTemporal= dcc.Graph(figure=ltLine)
    
            
            rtLine=px.line(currentDf[right].loc[minTf:maxTf:1])
            rtLine.update_layout(plot_bgcolor='#18242B', paper_bgcolor='#18242B',font_color='#95969A')
            lineChartRightFrontal= dcc.Graph(figure=rtLine)
        
            style={'height':'71.5vh','overflow-x': 'hidden','overflow-y': 'scroll'}
            
            return lineChartFrontal,lineChartCentral,lineChartParietal,lineChartOccipital,lineChartLeftTemporal,lineChartRightFrontal,GFPFig,style
    
            

### Run Server

In [43]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    app.run_server(port=4400,dev_tools_ui=False)


........................

Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated


.............................
----------------------------------------------------------------------
Ran 53 tests in 0.048s

OK


Dash app running on http://127.0.0.1:4400/
